## EE361M Introduction to Data Mining - Assignment #5
## Rohan Nagar, Wenyang Fu, Zuhair Parvez

# Question 1 (1+1+5+3+3+2=15pts) - Logistic Regression

In this question we will be predicting whether someone will have an affair! Yes - there is data on this. See below on how to import the data.
1. Convert naffairs to a binary variable hadAffair which is 1 if had an affair and zero otherwise
2. Split the data into training and test. Use 42 as random seed and use 1/3rd of the data for testing. Our y variable is hadAffair and X matrix includes all the other variables except naffairs.
3. Train a logistic regression with almost no regularization (pass l2 (ridge) to penalty and 1,000 to the C parameter which is the inverse of regularization strength lambda. This essentially does l2 regularization but applies very little weight to the penalty term) and report the confusion matrix on the test data. Also report the accuracy for the "no affairs" class, the affairs class, and the average per-class accuracy on the test data. Average per-class accuracy is described in this [post](http://blog.dato.com/how-to-evaluate-machine-learning-models-part-2a-classification-metrics).
4. Repeat step 3 except use l2 penalty with Cs of [.001, .01,0.1, 1]. You will want to use k-fold cross validation to select the best parameter. To evaluate which parameter is best, maximize the average per-class accuracy. To help with this task, check out [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and how to make your own [custom scorer](http://scikit-learn.org/stable/modules/model_evaluation.html).
5. Repeat question 4 except use l1 (i.e. Lasso) instead of l2 as the penalty type.
6. Which model produces the best average per-class accuracy? Why do you think this is the case? How do the models handle the different classes, and why is this so?

In [1]:
from pydataset import data
df = data('affairs')
print(len(df.values))  # number of instances
print(len(df.columns))   # number of features



601
18


# Answers

In [2]:
from sklearn.metrics import confusion_matrix as get_confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV

def custom_score_function(ground_truth, predictions):
    confusion_matrix = get_confusion_matrix(ground_truth, predictions)
    no_affairs_acc = confusion_matrix[0,0]/(sum(confusion_matrix[0]))
    affairs_acc = confusion_matrix[1,1]/(sum(confusion_matrix[1]))
    return (no_affairs_acc + affairs_acc) / 2

def print_metrics(ground_truth, predicted):
    confusion_matrix = get_confusion_matrix(Y_test, Y_predict)

    print("Confusion Matrix:")
    print(confusion_matrix)

    no_affairs_acc = confusion_matrix[0,0]/(sum(confusion_matrix[0]))
    print("No affairs accuracy: %0.02f" % no_affairs_acc)

    affairs_acc = confusion_matrix[1,1]/(sum(confusion_matrix[1]))
    print("Affairs accuracy: %0.02f" % affairs_acc)

    average_acc = (no_affairs_acc + affairs_acc) / 2
    print("Average accuracy: %0.02f" % average_acc)

scorer = make_scorer(custom_score_function, greater_is_better=True)

### Question 1.1 and 1.2

In [3]:
from sklearn.cross_validation import train_test_split

df['hadAffair'] = df['naffairs'] > 0

features = df.columns[1:-1] # naffairs is index 0 and hadAffair is -1
X = df[features].values
Y = df['hadAffair'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

### Question 1.3

In [4]:
from sklearn.linear_model import LogisticRegression

l2_classifier = LogisticRegression(C=1000, penalty='l2').fit(X_train, Y_train)
Y_predict = l2_classifier.predict(X_test)

print_metrics(Y_test, Y_predict)

Confusion Matrix:
[[136  10]
 [ 43  10]]
No affairs accuracy: 0.93
Affairs accuracy: 0.19
Average accuracy: 0.56


### Question 1.4

In [5]:
parameters = [{'C' : [.001, .01,0.1, 1], 'penalty':['l2']}]

classifier = GridSearchCV(LogisticRegression(), parameters, scoring=scorer)
classifier = classifier.fit(X_train, Y_train)

Y_predict = classifier.predict(X_test)

print_metrics(Y_test, Y_predict)

Confusion Matrix:
[[137   9]
 [ 43  10]]
No affairs accuracy: 0.94
Affairs accuracy: 0.19
Average accuracy: 0.56


### Question 1.5

In [6]:
parameters = [{'C' : [.001, .01,0.1, 1], 'penalty':['l1']}]

classifier = GridSearchCV(LogisticRegression(), parameters, scoring=scorer)
classifier = classifier.fit(X_train, Y_train)

Y_predict = classifier.predict(X_test)

print_metrics(Y_test, Y_predict)

Confusion Matrix:
[[138   8]
 [ 43  10]]
No affairs accuracy: 0.95
Affairs accuracy: 0.19
Average accuracy: 0.57


### Question 1.6

L2 and L1 (Lasso) logistic regression performed equally well on the Affairs class, but Lasso performed slightly better on the No Affairs class.

# Question 2 (2+3+2+3=10pts) - Support Vector Classifier

This question will continue to use the data from question 1 - including the training and test split data.
1. Fit a support vector classifier using the standard options on [sklearn's SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). Report the confusion matrix on the test data. Also report the accuracy for the no affairs class, the affairs class, and the average per-class accuracy (same as question 1).
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
3. Briefly discuss the effect of different  C,  kernel combinations.
4. Discuss your results from parts 1 and 2 and mention how they differ from Question 1's results.

# Answers

### Question 2.1

In [7]:
from sklearn.svm import SVC
classifier = SVC()
Y_predict = classifier.fit(X_train, Y_train).predict(X_test)
print_metrics(Y_test, Y_predict)

Confusion Matrix:
[[146   0]
 [ 53   0]]
No affairs accuracy: 1.00
Affairs accuracy: 0.00
Average accuracy: 0.50


### Question 2.2

In [8]:
search_params = [{'C':[0.001, 0.01, 0.1, 1, 5, 10, 100], 'kernel':['rbf','poly']}]

classifier = GridSearchCV(SVC(), search_params, scoring=scorer, cv=5)
classifier = classifier.fit(X_train, Y_train)

Y_predict = classifier.predict(X_test)

print_metrics(Y_test, Y_predict)

Confusion Matrix:
[[119  27]
 [ 37  16]]
No affairs accuracy: 0.82
Affairs accuracy: 0.30
Average accuracy: 0.56


### Question 2.3

The C parameter (our slack penalty) in the SVC model reflects a tradeoff between reducing misclassification and model simplicity. With a lower C value, the decision surface is smooth, but there are potentially more misclassifications. With a high C value, the model will try to classify all training data correctly by giving the model more freedom to select more support vectors. This parameter also represents the bias-variance tradeoff.

The kernel function allows us to map data into a high-dimensional space and compute the inner product - allowing us to find a hyperplane to seperate originally nonlinearally seperable data. Different kernels will try to map the data to the higher-dimensional space with different funtions. For example, we used RBF and Polynomial, and those functions can be viewed here:

https://en.wikipedia.org/wiki/Radial_basis_function_kernel

https://en.wikipedia.org/wiki/Polynomial_kernel

### Question 2.4

In part 1, we can see that the SVM classified everything as 'no affairs'. This obviously makes the 'no affairs' accuracy very good, but the average per-class accuracy is bad (0.50).

When we select the best C and kernel parameters, we get a much better SVM classifier. It looks similar to our answers in Question 1 in terms of the average per-class accuracy. One thing we can notice is that the 'affairs' accuracy actually performed a lot better than our Logistic Regression models in Q1, while our 'no affairs' accuracy did slightly worse. This is interesting to note, that our SVM was able to do a little better on classifing affairs.

# Question 3 (2+1+3+1+3=10pts) - Regression Trees

This question is very similar to homework 4 question 1. Except now we will be using regression trees and not classification trees, and you will be addressing a regression problem (i.e., the independent variable "price" will not be binarized).

For this question, we will be using the housing dataset (see code below). 

1. Convert driveway, recroom, fullbase, gashw, airco, and prefarea to numeric dummy variables (1 for yes, zero for no)
2. Split the data into training and testing with a random seed of 42 and keeping 1/3rd of the data for testing
2. Fit a [decision tree regressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) to predict price using all the data (your dummy variables plus bedrooms and bathrooms).
5. Report the root MSE on the test data.
6. How does the tree decide on a splitting point?

In [9]:
df = data('Housing')
print(len(df.values))  # number of instances
print(len(df.columns))   # number of features

546
12


# Answers

### Question 3.1

In [10]:
def yes_no_to_dummy(b):
    if type(b) is str: return 1 if b == 'yes' else 0
    else: return b

to_dummyify = ['driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'prefarea']
for feature in to_dummyify: df[feature] = df[feature].apply(yes_no_to_dummy)
df.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
1,42000.0,5850,3,1,2,1,0,1,0,0,1,0
2,38500.0,4000,2,1,1,1,0,0,0,0,0,0
3,49500.0,3060,3,1,1,1,0,0,0,0,0,0
4,60500.0,6650,3,1,2,1,1,0,0,0,0,0
5,61000.0,6360,2,1,1,1,0,0,0,0,0,0


### Question 3.2

In [11]:
features = ['lotsize', 'bedrooms', 'bathrms', 'stories', 'driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl', 'prefarea']
X_train, X_test, Y_train, Y_test = train_test_split(df[features], df['price'], test_size=0.33, random_state=42)

### Question 3.3

In [12]:
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor()
regr.fit(X_train, Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

### Question 3.4

In [13]:
from math import sqrt
from sklearn.metrics import mean_squared_error

score = mean_squared_error(Y_test, regr.predict(X_test))
print("Root MSE on test data: {}".format(sqrt(score)))

Root MSE on test data: 24902.383899840344


### Question 3.5

Makes split by choosing the best split that has the lowest MSE. It looks across all features and at all possible splits, where it calcuates the mean of the data on either side of the split and then calcuates the MSE based on that. The one with the lowest MSE will be the chosen split.

# Question 4 (2+5+3=10pts) - Support Vector Regression

This question will continue to use the data from question 3 - including the training and test split data.

1. Fit a support vector regression using the standard options on [sklearn's SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Report the root MSE.
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that minimizes MSE. Use 5 folds. Report the best model and the test root MSE.
4. Discuss your results from parts 1 and 2 and how they differ from Question 3 results.

# Answers

### Question 4.1

In [14]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)
Y_pred = SVR().fit(X_train_scaled, Y_train).predict(X_test_scaled)
score = mean_squared_error(Y_test, Y_pred)
print("Root MSE on test data: {}".format(sqrt(score)))

Root MSE on test data: 29002.714593474073


### Question 4.2

In [15]:
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
search_params = [{'C':[0.001, 0.01, 0.1, 1, 5, 10, 100], 'kernel':['rbf','poly']}]
clf = GridSearchCV(SVR(), search_params, scoring=mse_scorer, cv=5, n_jobs=-1)
clf.fit(X_train_scaled, Y_train)

Y_predict = clf.predict(X_test_scaled)

score = mean_squared_error(Y_test, Y_predict)
print("RMSE on test data: {}".format(sqrt(score)))

RMSE on test data: 25844.20711040443


### Question 4.3

RMSE for the regression tree was quite a bit better than the RMSE for the best Epsilon-Support Vector Regression (with feature scaling). Doing a grid search for good hyperparameters decreased the RMSE for Epsilon-Support Vector Regression quite a bit!

# Question 5 (3+2+2+2+2+2+2=15pts) - Random Forest

This question will also continue to use the data from Question 1.
1. Fit a random forest model grid searching over the following values: {'n_estimators':[10, 100, 1000], 'max_features': ['auto', 'sqrt', 'log2']}. Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
2. What do the n_estimators and max_features parameters do?
3. Report the features in order of importance based on the model used in part 1
4. Repeat question 1 using an AdaBoostClassifier and grid search over the following values: {'n_estimators':[50, 500, 5000], 'learning_rate': [.001, .01, .1]}
5. What does the learning_rate parameter do?
6. Report the features in order of importance based on the model used in part 4
7. Compare the results in part 1 and 4 and questions 2 and 3

# Question 5 Answers

In [16]:
df = data('affairs')
print(len(df.values))  # number of instances
print(len(df.columns))   # number of features
df['hadAffair'] = df['naffairs'] > 0

features = df.columns[1:-1] # naffairs is index 0 and hadAffair is -1
X = df[features].values
Y = df['hadAffair'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

601
18


### Question 5.1

In [17]:
from sklearn.ensemble import RandomForestClassifier

def custom_score_function(ground_truth, predictions):
    confusion_matrix = get_confusion_matrix(ground_truth, predictions)
    no_affairs_acc = confusion_matrix[0,0]/(sum(confusion_matrix[0]))
    affairs_acc = confusion_matrix[1,1]/(sum(confusion_matrix[1]))
    return (no_affairs_acc + affairs_acc) / 2
# make_scorer will wrap our avg. weighted per-class accuracy scorer for use
# with cross_val_score() and GridSearchCV
avg_class_score = make_scorer(custom_score_function, greater_is_better=True)
search_params =  [{'n_estimators':[10, 100, 1000], 'max_features': ['auto', 'sqrt', 'log2']}]
# Maximize avg. weighted per-class accuracy
clf = GridSearchCV(RandomForestClassifier(), search_params, scoring=avg_class_score, cv=5)
clf.fit(X_train, Y_train)

Y_predict = clf.predict(X_test)

print("Parameters for best RF model: {}".format(clf.best_params_))
print_metrics(Y_test, Y_predict)


Parameters for best RF model: {'max_features': 'sqrt', 'n_estimators': 100}
Confusion Matrix:
[[121  25]
 [ 40  13]]
No affairs accuracy: 0.83
Affairs accuracy: 0.25
Average accuracy: 0.54


### Question 5.2

I had help from (http://scikit-learn.org/stable/modules/ensemble.html#parameters)

n_estimators controls the number of decision trees in the forest. The more trees there are in the forest, the more accurate the answer will be, but the more computation time it will require. max_features controls the number of features that are considered in the random subset of features that are considered when splitting a node. A lower value for max_features will reduce variance, but increase bias.

### Question 5.3

In [18]:
features_by_importance = [(idx, val) for idx, val in sorted(enumerate(
                          clf.best_estimator_.feature_importances_), key=lambda x:-x[1])]
print(features_by_importance)
feature_names_by_importance = [df.columns[idx] for idx, _ in features_by_importance]
print(feature_names_by_importance)


[(0, 0.099130924465364381), (5, 0.0829224281537361), (2, 0.081706709422682114), (9, 0.065843027081769451), (6, 0.062939770896435335), (7, 0.060501684025759443), (14, 0.059672495394466264), (8, 0.058733627410223921), (16, 0.053821238229309537), (3, 0.051751161213939516), (15, 0.050106297710958578), (4, 0.049974912495475345), (1, 0.048569435069536662), (13, 0.047117058526842663), (12, 0.045497621883933581), (10, 0.041708403157128593), (11, 0.040003204862438607)]
['naffairs', 'hapavg', 'vryunhap', 'slghtrel', 'vryhap', 'antirel', 'yrsmarr3', 'notrel', 'yrsmarr5', 'unhap', 'yrsmarr4', 'avgmarr', 'kids', 'yrsmarr2', 'yrsmarr1', 'smerel', 'vryrel']


### Question 5.4

In [19]:
from sklearn.ensemble import AdaBoostClassifier
search_params =  [{'n_estimators':[50, 500, 5000], 'learning_rate': [.001, .01, .1]}]
# Maximize avg. weighted per-class accuracy
clf = GridSearchCV(AdaBoostClassifier(), search_params, scoring=avg_class_score, cv=5)
clf.fit(X_train, Y_train)

Y_predict = clf.predict(X_test)

print("Parameters for best AdaBoost model: {}".format(clf.best_params_))
print_metrics(Y_test, Y_predict)

Parameters for best AdaBoost model: {'n_estimators': 5000, 'learning_rate': 0.01}
Confusion Matrix:
[[135  11]
 [ 43  10]]
No affairs accuracy: 0.92
Affairs accuracy: 0.19
Average accuracy: 0.56


### Question 5.5
The contribution of each weak learner will be scaled by a factor of the learning rate. Decreasing learning_rate will improve test error, as long as n_estimators is increased to compensate.

### Question 5.6

In [20]:
features_by_importance = [(idx, val) for idx, val in sorted(enumerate(
                          clf.best_estimator_.feature_importances_), key=lambda x:-x[1])]
print(features_by_importance)
feature_names_by_importance = [df.columns[idx] for idx, _ in features_by_importance]
print(feature_names_by_importance)


[(12, 0.125), (10, 0.1008), (9, 0.0998), (2, 0.099000000000000005), (6, 0.096000000000000002), (1, 0.089599999999999999), (3, 0.077600000000000002), (11, 0.062799999999999995), (15, 0.056800000000000003), (13, 0.047199999999999999), (8, 0.0402), (16, 0.039600000000000003), (4, 0.0332), (0, 0.016400000000000001), (5, 0.016), (7, 0.0), (14, 0.0)]
['yrsmarr1', 'smerel', 'slghtrel', 'vryunhap', 'vryhap', 'kids', 'unhap', 'vryrel', 'yrsmarr4', 'yrsmarr2', 'notrel', 'yrsmarr5', 'avgmarr', 'naffairs', 'hapavg', 'antirel', 'yrsmarr3']


### Question 5.7

In question 1, our best accuracy (with logistic regression & L1 loss) was:
No affairs accuracy: 0.95
Affairs accuracy: 0.19
Average accuracy: 0.57

In question 2, our accuracy (with SVC) was:
No affairs accuracy: 0.82
Affairs accuracy: 0.30
Average accuracy: 0.56

In question 5.1, our accuracy (with RFs) was:
No affairs accuracy: 0.83
Affairs accuracy: 0.25
Average accuracy: 0.54

In question 5.4, our accuracy (with AdaBoost) was:
No affairs accuracy: 0.92
Affairs accuracy: 0.19
Average accuracy: 0.56


1. It seems that the logistic regression model with L1 loss had the highest average accuracy out of all of the models. It was also the best at classifying the No Affairs class. 

2. The SVC was the best at classifying the Affairs class. 

3. The RF model did only slightly better than the SVC on the no affairs class, and much worse on the affairs class. It did beat out the other two

4. Lastly, the AdaBoost model fell short of the logistic regression with L1 loss, but did have higher No affairs accuracy than 2 of the models.